# Converting prices to returns

1. Import the libraries

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

2. Download the data and keep the adjusted close price only:

In [2]:
df = yf.download('AAPL',
                      start='2000-01-01',
                      end='2010-12-31',
                      progress=False)

df = df.loc[:, ['Adj Close']]
df.rename(columns={'Adj Close':'adj_close'}, inplace=True)

3. Converted adjusted close prices to simple log returns:

In [3]:
df['simple_rtn'] = df.adj_close.pct_change()
df['log_rtn'] = np.log(df.adj_close/df.adj_close.shift(1))

4. Inspect the output

In [4]:
df.head()

,adj_close,simple_rtn,log_rtn
Date,,,
2000-01-03,0.854541,NaN,NaN
2000-01-04,0.782495,-0.084310,-0.088077
2000-01-05,0.793945,0.014633,0.014527
2000-01-06,0.725238,-0.086538,-0.090514
2000-01-07,0.759592,0.047369,0.046281


# There's more

1. Import the libraries and authenticate

In [5]:
import pandas as pd
import quandl

QUANDL_KEY = '6oGKxKQUWCBsryf3cVpQ' # replace your own API key
quandl.ApiConfig.api_key = QUANDL_KEY

2. Create a DataFrame with all possible dates and left join the prices on it: 

In [6]:
df_all_dates = pd.DataFrame(index=pd.date_range(start='1999-12-31',
                                                end='2010-12-31'))
df = df_all_dates.join(df[['adj_close']], how='left') \
    .fillna(method='ffill') \
        .asfreq('M')

3. Download inflation data from Quandl:

In [7]:
df_cpi = quandl.get(dataset='RATEINF/CPI_USA',
                    start_date='1999-12-01',
                    end_date='2010-12-31')
df_cpi.rename(columns={'Value':'cpi'}, inplace=True)

4. Merge inflation data to prices:

In [8]:
df_merged = df.join(df_cpi, how='left')

5. Calculate simple returns and inflation rate:

In [9]:
df_merged['simple_rtn'] = df_merged.adj_close.pct_change()
df_merged['inflation_rate'] = df_merged.cpi.pct_change()

6. Adjust returns for inflation:

In [10]:
df_merged['real_rtn'] = (df_merged.simple_rtn + 1) / (df_merged.inflation_rate + 1) - 1
df_merged.head()

,adj_close,cpi,simple_rtn,inflation_rate,real_rtn
1999-12-31,NaN,168.3,NaN,NaN,NaN
2000-01-31,0.792036,168.8,NaN,0.002971,NaN
2000-02-29,0.875058,169.8,0.104820,0.005924,0.098314
2000-03-31,1.036805,171.2,0.184842,0.008245,0.175153
2000-04-30,0.947104,171.3,-0.086517,0.000584,-0.087050
